## Resnet34

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git

# Land Use Land Cover with U-Net
## Importing Libraries

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import classification_models.tfkeras as tfkeras
from classification_models.tfkeras import Classifiers
from tensorflow.keras.metrics import MeanIoU, Precision, Recall
import random
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Get an understanding by looking at a few random images and masks
train_img_dir = "/kaggle/input/lulc-data/data_for_training_and_testing/train/images/"
train_mask_dir = "/kaggle/input/lulc-data/data_for_training_and_testing/train/masks/"

img_list = os.listdir(train_img_dir)
msk_list = os.listdir(train_mask_dir)

num_images = len(os.listdir(train_img_dir))

img_num = random.randint(0, num_images-1)

img_for_plot = cv2.imread(train_img_dir + img_list[img_num], 1)
img_for_plot = cv2.cvtColor(img_for_plot, cv2.COLOR_BGR2RGB)

mask_for_plot = cv2.imread(train_mask_dir + msk_list[img_num], 0)

plt.figure(figsize=(12, 8))
plt.subplot(121)
plt.imshow(img_for_plot)
plt.title('Image')
plt.subplot(122)
plt.imshow(mask_for_plot, cmap='gray')
plt.title('Mask')
plt.show()

## Defining Data Generators ensuring correct image loading and shuffling

In [ ]:
# Define Generator for images and masks
seed = 24
batch_size = 16
n_classes = 4

from keras.utils import to_categorical

# Use this to preprocess input for transfer learning
BACKBONE = 'resnet34'
ResNet34, preprocess_input = Classifiers.get(BACKBONE)

In [ ]:
# Define a function to perform additional preprocessing
def preprocess_data(img, mask, num_class):
    # Normalize images to [0, 1] manually (compatible with TensorFlow)
    img = img.astype(np.float32) / 255.0
    img = preprocess_input(img)  # Preprocess based on the pretrained backbone...
    # Ensure mask values are within [0, num_class-1]
    mask = np.clip(mask, 0, num_class - 1).astype(np.uint8)
    mask = to_categorical(mask, num_class)
    return (img, mask)

# Define a custom generator for semantic segmentation
def custom_generator(img_dir, mask_dir, batch_size, num_class):
    img_list = os.listdir(img_dir)
    mask_list = os.listdir(mask_dir)
    if len(img_list) != len(mask_list):
        raise ValueError("Mismatch between image and mask counts")
    while True:
        for start in range(0, len(img_list), batch_size):
            img_batch = []
            mask_batch = []
            end = min(start + batch_size, len(img_list))
            for img_path, mask_path in zip(img_list[start:end], mask_list[start:end]):
                img = cv2.imread(os.path.join(img_dir, img_path))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                mask = cv2.imread(os.path.join(mask_dir, mask_path), 0)
                if img is None or mask is None:
                    print(f"Failed to load: {img_path} or {mask_path}")
                    continue
                img, mask = preprocess_data(img, mask, num_class)
                img_batch.append(img)
                mask_batch.append(mask)
            yield np.array(img_batch), np.array(mask_batch)

train_img_gen = custom_generator(train_img_dir, train_mask_dir, batch_size, n_classes)
val_img_dir = "/kaggle/input/lulc-data/data_for_training_and_testing/val/images/"
val_mask_dir = "/kaggle/input/lulc-data/data_for_training_and_testing/val/masks/"
val_img_gen = custom_generator(val_img_dir, val_mask_dir, batch_size, n_classes)

## Data Verification

In [ ]:
# Make sure the generator is working
num_samples = 2  
for _ in range(num_samples // batch_size + 1):
    try:
        x, y = next(train_img_gen)
        for i in range(min(num_samples, x.shape[0])):
            image = x[i]
            mask = np.argmax(y[i], axis=2)
            plt.figure(figsize=(12, 5))
            plt.subplot(121)
            plt.imshow(image)
            plt.title(f' Sample Training Image {i+1}')
            plt.subplot(122)
            plt.imshow(mask, cmap='gray')
            plt.title(f'Sample Training Mask {i+1}')
            plt.show()
    except StopIteration:
        break

for _ in range(num_samples // batch_size + 1):
    try:
        x_val, y_val = next(val_img_gen)
        for i in range(min(num_samples, x_val.shape[0])):
            image = x_val[i]
            mask = np.argmax(y_val[i], axis=2)
            plt.figure(figsize=(12, 5))
            plt.subplot(121)
            plt.imshow(image)
            plt.title(f' Sample Validation Image {i+1}')
            plt.subplot(122)
            plt.imshow(mask, cmap='gray')
            plt.title(f'Sample Validation Mask {i+1}')
            plt.show()
    except StopIteration:
        break

## Defining and Training U-Net Model

In [ ]:
# Define the model metrics and load model
num_train_imgs = len(os.listdir(train_img_dir))
num_val_images = len(os.listdir(val_img_dir))
steps_per_epoch = num_train_imgs // batch_size
val_steps_per_epoch = num_val_images // batch_size

IMG_HEIGHT = x.shape[1]
IMG_WIDTH = x.shape[2]
IMG_CHANNELS = x.shape[3]

n_classes = 4

In [ ]:
# Define the U-Net model with ResNet34 as backbone (encoder) and custom decoder
def unet_resnet34(input_shape, n_classes):
    # Load ResNet34 backbone as encoder
    backbone = ResNet34(input_shape=input_shape, weights='imagenet', include_top=False)
    
    # Extract layers from ResNet34 for the encoder
    conv1 = backbone.get_layer("relu0").output  # Initial block after input
    conv2 = backbone.get_layer("stage2_unit1_relu1").output  # After stage 2
    conv3 = backbone.get_layer("stage3_unit1_relu1").output  # After stage 3
    conv4 = backbone.get_layer("stage4_unit1_relu1").output  # After stage 4
    conv5 = backbone.get_layer("relu1").output  # Final block before top

    # Decoder (U-Net style)
    up6 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv5)
    up6 = tf.keras.layers.Concatenate()([up6, conv4])
    conv6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(up6)
    conv6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(conv6)

    up7 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv6)
    up7 = tf.keras.layers.Concatenate()([up7, conv3])
    conv7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up7)
    conv7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(conv7)

    up8 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv7)
    up8 = tf.keras.layers.Concatenate()([up8, conv2])
    conv8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up8)
    conv8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(conv8)

    up9 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv8)
    up9 = tf.keras.layers.Concatenate()([up9, conv1])
    conv9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up9)
    conv9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(conv9)

    up10 = tf.keras.layers.UpSampling2D(size=(2, 2))(conv9)
    conv10 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up10)
    conv10 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(conv10)

    outputs = tf.keras.layers.Conv2D(n_classes, (1, 1), activation='softmax')(conv10)
    model = tf.keras.Model(inputs=[backbone.input], outputs=[outputs])
    return model

# Create and compile the model
model = unet_resnet34(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), n_classes=n_classes)
model.compile('Adam', loss='categorical_crossentropy', metrics=[MeanIoU(num_classes=n_classes)])

print(model.summary())
print(model.input_shape)# Define the model metrics and load model
num_train_imgs = len(os.listdir(train_img_dir))
num_val_images = len(os.listdir(val_img_dir))
steps_per_epoch = num_train_imgs // batch_size
val_steps_per_epoch = num_val_images // batch_size

IMG_HEIGHT = x.shape[1]
IMG_WIDTH = x.shape[2]
IMG_CHANNELS = x.shape[3]

n_classes = 4

In [ ]:
# Train the model
history = model.fit(train_img_gen,
          steps_per_epoch=steps_per_epoch,
          epochs=100,
          verbose=1,
          validation_data=val_img_gen,
          validation_steps=val_steps_per_epoch)

model.save('landcover_RESNET_backbone_batch16.hdf5')

In [ ]:
model.save('landcover_RESNET_backbone_batch16.hdf5')

In [ ]:
# Plot the training and validation IoU and loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['mean_io_u']
val_acc = history.history['val_mean_io_u']

plt.plot(epochs, acc, 'y', label='Training IoU')
plt.plot(epochs, val_acc, 'r', label='Validation IoU')
plt.title('Training and validation IoU')
plt.xlabel('Epochs')
plt.ylabel('IoU')
plt.legend()
plt.show()

# Predictions and Visualisation
## 1.Predictions made on Original Dataset

In [ ]:
# Load and evaluate the model
from keras.models import load_model

model = load_model("landcover_RESNET_backbone_batch16.hdf5", compile=False)

# Test generator using validation data
test_image_batch, test_mask_batch = val_img_gen.__next__()

# Convert categorical to integer for visualization and IoU calculation
test_mask_argmaxed = np.argmax(test_mask_batch, axis=3)
test_pred_batch = model.predict(test_image_batch)
test_pred_argmaxed = np.argmax(test_pred_batch, axis=3)

n_classes = 4
IOU_keras = MeanIoU(num_classes=n_classes)
IOU_keras.update_state(test_pred_argmaxed, test_mask_argmaxed)
print("Mean IoU (Validation) =", IOU_keras.result().numpy())

In [ ]:
# View a few images, masks, and corresponding predictions from validation
for i in range(5):  
    # Fetch a new batch for each visualization to ensure randomness
    test_image_batch, test_mask_batch = val_img_gen.__next__()
    test_pred_batch = model.predict(test_image_batch)
    test_mask_argmaxed = np.argmax(test_mask_batch, axis=3)
    test_pred_argmaxed = np.argmax(test_pred_batch, axis=3)
    
    img_num = random.randint(0, test_image_batch.shape[0]-1)
    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title(f'Sample Image {i+1}')
    plt.imshow(test_image_batch[img_num])
    plt.subplot(232)
    plt.title(f'Sample Mask {i+1}')
    plt.imshow(test_mask_argmaxed[img_num])
    plt.subplot(233)
    plt.title(f' Predicted Image {i+1}')
    plt.imshow(test_pred_argmaxed[img_num])
    plt.show()

## 2.Predictions made on Test Dataset

In [ ]:
# Test generator using new testing dataset
test_img_dir = "/kaggle/input/lulc-data/data_for_training_and_testing/test/images/"
test_mask_dir = "/kaggle/input/lulc-data/data_for_training_and_testing/test/masks/"
new_test_img_gen = custom_generator(test_img_dir, test_mask_dir, batch_size, n_classes)

In [ ]:
# Evaluate on new test dataset
new_test_image_batch, new_test_mask_batch = new_test_img_gen.__next__()

# Convert categorical to integer for visualization and IoU calculation for new test
new_test_mask_argmaxed = np.argmax(new_test_mask_batch, axis=3)
new_test_pred_batch = model.predict(new_test_image_batch)
new_test_pred_argmaxed = np.argmax(new_test_pred_batch, axis=3)

n_classes = 4
IOU_keras = MeanIoU(num_classes=n_classes)
IOU_keras.update_state(new_test_pred_argmaxed, new_test_mask_argmaxed)
print("Mean IoU (New Test) =", IOU_keras.result().numpy())

In [ ]:
# Visualize 5 random samples from new test dataset
for i in range(5):  
    new_test_image_batch, new_test_mask_batch = new_test_img_gen.__next__()
    new_test_pred_batch = model.predict(new_test_image_batch)
    new_test_mask_argmaxed = np.argmax(new_test_mask_batch, axis=3)
    new_test_pred_argmaxed = np.argmax(new_test_pred_batch, axis=3)
    
    img_num = random.randint(0, new_test_image_batch.shape[0]-1)
    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title(f'New Testing Image {i+1}')
    plt.imshow(new_test_image_batch[img_num])
    plt.subplot(232)
    plt.title(f'New Testing Mask {i+1}')
    plt.imshow(new_test_mask_argmaxed[img_num])
    plt.subplot(233)
    plt.title(f' New Predicted Image {i+1}')
    plt.imshow(new_test_pred_argmaxed[img_num])
    plt.show()

## Comparing the performance between Original Dataset and New Dataset

In [ ]:
def calculate_metrics(y_true, y_pred):
    precision = Precision()(y_true, y_pred).numpy()
    recall = Recall()(y_true, y_pred).numpy()
    iou = MeanIoU(num_classes=n_classes)(y_true, y_pred).numpy()
    return {'IoU': iou, 'Precision': precision, 'Recall': recall}

val_metrics = calculate_metrics(test_mask_argmaxed, test_pred_argmaxed)
test_metrics = calculate_metrics(new_test_mask_argmaxed, new_test_pred_argmaxed)

metrics_df = pd.DataFrame({
    'Dataset': ['Validation', 'Test'],
    'IoU': [val_metrics['IoU'], test_metrics['IoU']],
    'Precision': [val_metrics['Precision'], test_metrics['Precision']],
    'Recall': [val_metrics['Recall'], test_metrics['Recall']]
})
print("\n==================== PERFORMANCE METRICS COMPARISON ====================")
print("Metric    |   Validation |        Test |       Diff")
print("---------------------------------------------------------")
print(f"IoU       |    {metrics_df.loc[0, 'IoU']:.4f} |    {metrics_df.loc[1, 'IoU']:.4f} |     {(metrics_df.loc[1, 'IoU'] - metrics_df.loc[0, 'IoU']):.4f}")
print(f"Precision |    {metrics_df.loc[0, 'Precision']:.4f} |    {metrics_df.loc[1, 'Precision']:.4f} |     {(metrics_df.loc[1, 'Precision'] - metrics_df.loc[0, 'Precision']):.4f}")
print(f"Recall    |    {metrics_df.loc[0, 'Recall']:.4f} |    {metrics_df.loc[1, 'Recall']:.4f} |     {(metrics_df.loc[1, 'Recall'] - metrics_df.loc[0, 'Recall']):.4f}")

In [ ]:
# Visualize performance comparison (Original vs. New Test Dataset)
val_iou = IOU_keras.result().numpy()
val_loss = np.mean(tf.keras.losses.categorical_crossentropy(test_mask_batch, test_pred_batch))
new_test_iou = IOU_keras.result().numpy()
new_test_loss = np.mean(tf.keras.losses.categorical_crossentropy(new_test_mask_batch, new_test_pred_batch))

metrics = ['Loss', 'Mean IoU']
original_values = [val_loss, val_iou]
new_values = [new_test_loss, new_test_iou]

plt.figure(figsize=(10, 6))
x = np.arange(len(metrics))
width = 0.35

plt.bar(x - width/2, original_values, width, label='Original (Validation)', color='#B22222')
plt.bar(x + width/2, new_values, width, label='New Test', color='#D3D3D3')
plt.xlabel('Metrics')
plt.ylabel('Value')
plt.title('Performance Comparison: Original vs. New Test Dataset')
plt.xticks(x, metrics)
plt.legend()
plt.show()

## Post Training Visualisations

#### 3. Confusion Matrix Analysis:
    Presents heatmaps showing prediction errors for validation and test datasets, revealing which classes the model confuses.

#### 4. Prediction Visualization:
    Visualizes random images, true masks, predicted masks, and errors from validation and test datasets to inspect model predictions visually.

#### 5. Class Distribution Analysis:
    Plots bar charts of class counts (Background, Buildings, Woodlands, Water) in validation and test datasets to identify data imbalances.

#### 6. Probability Calibration Analysis:
    Draws density curves of predicted probabilities for each class in validation and test datasets to assess model confidence.

#### 7. Detailed Performance Comparison:
    Creates a bar chart comparing IoU, Precision, and Recall between validation and test datasets to evaluate generalization.

#### 8. Per-Class IoU Analysis:
    Displays bar charts of IoU for each class (0–3) in validation and test datasets to highlight individual class performance.

In [ ]:
# 3. Confusion Matrix Analysis

def plot_confusion_matrix(y_true, y_pred, title):
    cm = confusion_matrix(y_true.flatten(), y_pred.flatten())
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=[f'Class {i}' for i in range(n_classes)],
                yticklabels=[f'Class {i}' for i in range(n_classes)])
    plt.title(f'Confusion Matrix ({title})')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

plot_confusion_matrix(test_mask_argmaxed, test_pred_argmaxed, 'Validation Set')
plot_confusion_matrix(new_test_mask_argmaxed, new_test_pred_argmaxed, 'Test Set')

In [ ]:
# 4. Prediction Visualization

def plot_predictions(images, masks, preds, num_samples=3):
    indices = np.random.choice(len(images), num_samples)
    plt.figure(figsize=(15, num_samples*4))
    
    for i, idx in enumerate(indices):
        plt.subplot(num_samples, 4, i*4+1)
        plt.imshow(images[idx])
        plt.title(f'Image {idx}')
        plt.axis('off')
        
        plt.subplot(num_samples, 4, i*4+2)
        plt.imshow(masks[idx])
        plt.title('True Mask')
        plt.axis('off')
        
        plt.subplot(num_samples, 4, i*4+3)
        plt.imshow(preds[idx])
        plt.title('Predicted Mask')
        plt.axis('off')
        
        plt.subplot(num_samples, 4, i*4+4)
        error = np.where(masks[idx] != preds[idx], 1, 0)
        plt.imshow(error)
        plt.title(f'Errors: {error.sum()}')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

print("\nValidation Set Predictions:")
plot_predictions(test_image_batch, test_mask_argmaxed, test_pred_argmaxed)

print("\nTest Set Predictions:")
plot_predictions(new_test_image_batch, new_test_mask_argmaxed, new_test_pred_argmaxed)

In [ ]:
# 5. Class Distribution Analysis

def plot_class_distribution(*masks_list, titles):
    plt.figure(figsize=(15, 5))
    colors = sns.color_palette('husl', len(masks_list))
    
    for i, masks in enumerate(masks_list):
        counts = np.bincount(masks.flatten(), minlength=n_classes)
        plt.subplot(1, len(masks_list), i+1)
        plt.bar(range(n_classes), counts, color=colors[i])
        plt.title(titles[i])
        plt.xlabel('Class')
        plt.ylabel('Count')
        plt.xticks(range(n_classes))
    plt.tight_layout()
    plt.show()

plot_class_distribution(
    test_mask_argmaxed, new_test_mask_argmaxed,
    titles=['Validation Class Distribution', 'Test Class Distribution']
)

In [ ]:
# 6. Probability Calibration Analysis

def plot_probability_distribution(predictions, title):
    plt.figure(figsize=(12, 6))
    for class_idx in range(n_classes):
        sns.kdeplot(predictions[..., class_idx].flatten(), 
                    label=f'Class {class_idx}', linewidth=2)
    plt.title(f'Class Probability Distribution ({title})')
    plt.xlabel('Predicted Probability')
    plt.ylabel('Density')
    plt.legend()
    plt.show()

print("\nValidation Probability Distributions:")
plot_probability_distribution(test_pred_batch, 'Validation')

print("\nTest Probability Distributions:")
plot_probability_distribution(new_test_pred_batch, 'Test')

In [ ]:
# 7. Detailed Performance Comparison

def plot_metric_comparison(val_metrics, test_metrics):
    metrics = ['IoU', 'Precision', 'Recall']
    val_values = [val_metrics[m] for m in metrics]
    test_values = [test_metrics[m] for m in metrics]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    plt.figure(figsize=(10, 6))
    plt.bar(x - width/2, val_values, width, label='Validation', color='navy')
    plt.bar(x + width/2, test_values, width, label='Test', color='#87CEEB')
    
    plt.title('Performance Metric Comparison')
    plt.ylabel('Score')
    plt.xticks(x, metrics)
    plt.ylim(0, 1)
    plt.legend()
    plt.show()

plot_metric_comparison(val_metrics, test_metrics)

In [ ]:
# 8. Per-Class IoU Analysis

def plot_class_iou(y_true, y_pred, title):
    class_iou = []
    for class_id in range(n_classes):
        iou = MeanIoU(num_classes=n_classes)
        iou.update_state(y_true == class_id, y_pred == class_id)
        class_iou.append(iou.result().numpy())
    
    plt.figure(figsize=(10, 6))
    colors = sns.color_palette('viridis', n_classes)
    plt.bar(range(n_classes), class_iou, color=colors)
    plt.title(f'Per-Class IoU ({title})')
    plt.xlabel('Class')
    plt.ylabel('IoU')
    plt.xticks(range(n_classes))
    plt.ylim(0, 1)
    

print("\nValidation Set Class IoU:")
plot_class_iou(test_mask_argmaxed, test_pred_argmaxed, 'Validation')

print("\nTest Set Class IoU:")
plot_class_iou(new_test_mask_argmaxed, new_test_pred_argmaxed, 'Test')